In [2]:
import sys

import numpy as np
import pandas as pd
import datetime
import time

# music api imports
import spotipy
import spotipy.util as util
import billboard
import pygn
import pylast

import requests
import re
from bs4 import BeautifulSoup
from time import sleep
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print f.__name__, 'took', end - start, 'seconds'
        return result
    return f_timer

In [3]:
# Setting up tokens and authorization - hidden for github, use own tokens

# Spotify
token = util.prompt_for_user_token(<INSERT SPOTIFY USERNAME>,
                                   client_id=<INSERT SPOTIFY CLIENT ID>,
                                   client_secret=<INSERT SPOTIFY CLIENT SECRET>,
                                   redirect_uri=<INSERT REDIRECT URI>)

spotify = spotipy.Spotify(auth=token)

# Gracenote
clientid=<INSERT GRACENOTE CLIENTID>
userid = pygn.register(clientid)

# last.fm
apikey = <INSERT LASTFM API KEY>
apisecret = <INSERT LASTFM API SECRET>
username = <INSERT YOUR USERNAME>
password_hash = pylast.md5(<INSERT YOUR PASSWORD>)

lastfm = pylast.LastFMNetwork(api_key = apikey, api_secret = apisecret,
                               username = username, password_hash = password_hash)

In [4]:
songs = pd.read_pickle('data/spotify_songs.pkl')

In [5]:
songs

,artist,explicit,id,popularity,title,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration_ms,loudness,valence,mode
0,Nirvana,False,3FUsMXBxA4V7eUwQ7B0HQO,48,Love Buzz,0.885000,0.169,138.153,0.0354,0.000032,0.196,4,0.47,3.0,215120.0,-7.224,0.78,1.0
1,The Presidents Of The United States Of America,False,2JdzB67NvIa90K4gEZPLeX,58,Lump,0.873000,0.165,142.726,0.0386,0.004680,0,4,0.499,6.0,134200.0,-3.902,0.869,1.0
2,Stanton Warriors,False,0zXLMZUCYZCx8Bq4jCr174,30,So Sweet,0.936000,0.118,127.997,0.0425,0.000531,0.594,4,0.715,4.0,201337.0,-6.127,0.106,0.0
3,Natalie Marchenko,False,14w7n7gi635VCb5f7t2OQ5,37,License To Chill - Pacha Sax Lounge RMX,0.511000,0.24,171.963,0.0302,0.262000,0.837,4,0.581,5.0,317811.0,-8.199,0.399,0.0
4,Robyn,False,01NJd6s7Kyn6NSsPp503sh,28,Be Mine! - Live At The Cherrytree House,0.249000,0.114,116.599,0.0451,0.955000,0,4,0.533,0.0,272053.0,-8.532,0.513,0.0
5,Pearl Jam,False,6qgpO647NPBUvBkm56vV7d,5,Black,0.548000,0.179,76.020,0.0287,0.294000,0,4,0.376,4.0,343667.0,-10.580,0.233,0.0
6,Sam Hunt,False,3wx2kQWPn9p5UppQbNhPAk,67,Leave The Night On,0.953000,0.349,171.971,0.0624,0.099600,0,4,0.516,9.0,192160.0,-3.828,0.849,1.0
7,George Carlin,False,5DuxYjcuebFPontojL1NEE,0,War Pictures,0.546000,0.147,72.940,0.961,0.921000,0,3,0.611,7.0,222200.0,-10.444,0.491,1.0
8,Kaya May,False,4eg3wsiTrPa7KpwVai2VY7,56,"Hands to Myself, Love Yourself, Perfect (Acous...",0.445000,0.113,110.056,0.05,0.057500,0,4,0.702,9.0,226893.0,-9.693,0.326,0.0
9,Notaker,False,341ShRaeysVdxLetlI391K,56,Infinite,0.669000,0.108,99.987,0.0483,0.093700,0.82,4,0.633,1.0,355200.0,-8.124,0.0384,0.0


In [6]:
# Get track info from gracenote
# NOTE: Gracenote API has built-in fuzzy matching to artist and track.
from collections import defaultdict

def get_gn_multiple(search, dictionary, item):
    '''
    Helper function to get multiple items within Gracenote record
    '''
    for i in search[item].iteritems():
        dictionary[item + '_' + i[0]] = i[1]['TEXT']

def get_gn(artist, track):
    '''
    Gets artist and track information from Gracenote
    '''
    gn_dict = defaultdict(list)
    gn_info = pygn.search(clientid, userid, artist=artist, track=track)

    gn_dict['gnid'] = gn_info['track_gnid']

    # artist specific info
    for a in ['artist_origin', 'artist_type', 'artist_era']:
        get_gn_multiple(gn_info, gn_dict, a)
    # track specific info
    for s in ['genre', 'mood', 'tempo']: # can potentially drop 'tempo' since Spotify has already captured this
        get_gn_multiple(gn_info, gn_dict, s)      

    return dict(gn_dict)

In [7]:
@timefunc
def append_gn_df(df):
    '''
    Creates dataframe with artist and track information from Gracenote
    '''
    d = {}
    for index, row in df.iterrows():
        try:
            d[row['id']] = get_gn(row['artist'], row['title'])
        except:
            pass
    gn_df = pd.DataFrame(d).transpose().reset_index()
    gn_df = gn_df.rename(columns = {'index' : 'id'})
    return gn_df

In [14]:
@timefunc
def chunk_gn_df(df, start, stop):
    '''
    Gets Gracenote dataframe from dataframe with spotify IDs and merges them, indexing based on Spotify dataframe
    '''
    full_songs = pd.DataFrame()
    for i in range(start, stop, 100): 
        gn = append_gn_df(df.iloc[i], attach = False)
        data = pd.merge(df, gn, on='id')
        full_songs = full_songs.append(data, ignore_index=True)
        full_songs.to_pickle('data/full_songs_df_' + str(i[-1]) + '.pkl')
        sleep(.1)

### Testing functions

In [8]:
test = songs[:10].copy()

In [203]:
test_gn_df = append_gn_df(test)

append_gn_df took 6.28788208961 seconds


In [9]:
test2 = songs[:100].copy()

In [35]:
test_gn_df2 = append_gn_df(test2)

append_gn_df took 78.8128910065 seconds


In [10]:
test3 = songs[:1000].copy()

In [13]:
test = chunk_gn_df(test3)

append_gn_df took 79.7802000046 seconds
append_gn_df took 86.6563389301 seconds
append_gn_df took 65.2759389877 seconds
append_gn_df took 79.6878931522 seconds
append_gn_df took 76.4993541241 seconds
append_gn_df took 90.4260671139 seconds
append_gn_df took 72.7159290314 seconds
append_gn_df took 76.5112099648 seconds
append_gn_df took 94.4111959934 seconds
append_gn_df took 79.794656992 seconds
chunk_gn_df took 802.02742815 seconds


## Batch Gracenote API Calls

In [15]:
## Can be called iteratively; manual process and assignemnt to variables not necessary
full_songs1999 = chunk_gn_df(songs, 1000, 2000)

append_gn_df took 67.4224410057 seconds
append_gn_df took 72.3883001804 seconds
append_gn_df took 79.8355090618 seconds
append_gn_df took 78.1213560104 seconds
append_gn_df took 80.340474844 seconds
append_gn_df took 64.7614121437 seconds
append_gn_df took 79.3378539085 seconds
append_gn_df took 77.4290459156 seconds
append_gn_df took 68.3279361725 seconds
append_gn_df took 84.0087909698 seconds
chunk_gn_df took 753.381345987 seconds


In [17]:
full_songs2999 = chunk_gn_df(songs, 2000, 3000)

Must query with at least one field (artist, album, track, toc)
append_gn_df took 84.309497118 seconds
append_gn_df took 72.887706995 seconds
append_gn_df took 96.2273700237 seconds
append_gn_df took 132.650071144 seconds
append_gn_df took 85.8785710335 seconds
append_gn_df took 88.1040289402 seconds
append_gn_df took 77.864866972 seconds
append_gn_df took 86.2918081284 seconds
append_gn_df took 69.6620688438 seconds
append_gn_df took 68.9744520187 seconds
chunk_gn_df took 864.217645884 seconds


In [19]:
full_songs3999 = chunk_gn_df(songs, 3000, 4000)

append_gn_df took 78.3683691025 seconds
append_gn_df took 79.9486739635 seconds
append_gn_df took 72.4678637981 seconds
append_gn_df took 78.1993000507 seconds
append_gn_df took 72.6659281254 seconds
append_gn_df took 76.5402870178 seconds
append_gn_df took 78.0043098927 seconds
append_gn_df took 73.5283191204 seconds
append_gn_df took 75.2775919437 seconds
append_gn_df took 74.0675299168 seconds
chunk_gn_df took 760.451182127 seconds


In [21]:
full_songs4999 = chunk_gn_df(songs, 4000, 5000)

append_gn_df took 76.9578900337 seconds
append_gn_df took 75.7942149639 seconds
append_gn_df took 73.5237259865 seconds
append_gn_df took 77.3375658989 seconds
append_gn_df took 82.8529810905 seconds
append_gn_df took 67.162250042 seconds
Must query with at least one field (artist, album, track, toc)
append_gn_df took 72.9738769531 seconds
append_gn_df took 71.6249399185 seconds
Must query with at least one field (artist, album, track, toc)
append_gn_df took 79.046284914 seconds
append_gn_df took 71.738476038 seconds
chunk_gn_df took 750.387325048 seconds


In [23]:
full_songs5999 = chunk_gn_df(songs, 5000, 6000)

append_gn_df took 95.8282520771 seconds
append_gn_df took 69.6465280056 seconds
append_gn_df took 83.172315836 seconds
append_gn_df took 78.8234639168 seconds
append_gn_df took 80.6341700554 seconds
Must query with at least one field (artist, album, track, toc)
append_gn_df took 68.7922978401 seconds
append_gn_df took 77.1774830818 seconds
append_gn_df took 83.2521080971 seconds
append_gn_df took 69.8379299641 seconds
append_gn_df took 70.1823079586 seconds
chunk_gn_df took 778.720597029 seconds


In [25]:
full_songs6999 = chunk_gn_df(songs, 6000, 7000)

append_gn_df took 86.7909920216 seconds
append_gn_df took 78.6328549385 seconds
append_gn_df took 78.0821502209 seconds
append_gn_df took 74.4237539768 seconds
append_gn_df took 69.9132301807 seconds
append_gn_df took 68.4916939735 seconds
append_gn_df took 79.7115499973 seconds
append_gn_df took 76.0548529625 seconds
append_gn_df took 83.470785141 seconds
append_gn_df took 80.4534299374 seconds
chunk_gn_df took 777.427739143 seconds


In [27]:
full_songs7999 = chunk_gn_df(songs, 7000, 8000)

append_gn_df took 87.3198399544 seconds
append_gn_df took 71.2444541454 seconds
append_gn_df took 64.7104129791 seconds
append_gn_df took 91.5900039673 seconds
append_gn_df took 78.8001978397 seconds
append_gn_df took 81.8212971687 seconds
append_gn_df took 78.2230608463 seconds
append_gn_df took 84.765859127 seconds
append_gn_df took 82.0997200012 seconds
append_gn_df took 86.5055379868 seconds
chunk_gn_df took 808.462280035 seconds


In [29]:
full_songs8999 = chunk_gn_df(songs, 8000, 9000)

append_gn_df took 77.7793679237 seconds
append_gn_df took 73.5890009403 seconds
append_gn_df took 72.8890779018 seconds
append_gn_df took 76.9375469685 seconds
append_gn_df took 87.3194820881 seconds
append_gn_df took 72.121778965 seconds
append_gn_df took 78.8284990788 seconds
append_gn_df took 88.3182590008 seconds
append_gn_df took 64.0786280632 seconds
append_gn_df took 74.8370850086 seconds
chunk_gn_df took 768.161731958 seconds


In [31]:
full_songs9999 = chunk_gn_df(songs, 9000, 10000)

append_gn_df took 76.7856390476 seconds
append_gn_df took 78.7005288601 seconds
append_gn_df took 78.9515731335 seconds
append_gn_df took 75.6537179947 seconds
append_gn_df took 84.0704278946 seconds
append_gn_df took 72.4143879414 seconds
append_gn_df took 125.271933079 seconds
append_gn_df took 153.4139359 seconds
append_gn_df took 119.485553026 seconds
append_gn_df took 308.530214071 seconds
chunk_gn_df took 1174.64778805 seconds


In [33]:
full_songs10999 = chunk_gn_df(songs, 10000, 11000)

append_gn_df took 131.71905899 seconds
append_gn_df took 96.9787621498 seconds
append_gn_df took 146.209033966 seconds
append_gn_df took 351.851246119 seconds
append_gn_df took 154.008980989 seconds
append_gn_df took 105.124425888 seconds
append_gn_df took 230.586303949 seconds
append_gn_df took 112.298628092 seconds
append_gn_df took 85.2859518528 seconds
append_gn_df took 110.163626194 seconds
chunk_gn_df took 1525.60910511 seconds


In [35]:
full_songs11999 = chunk_gn_df(songs, 11000, 12000)

append_gn_df took 98.8968849182 seconds
append_gn_df took 97.7458968163 seconds
append_gn_df took 91.483907938 seconds
append_gn_df took 107.240098953 seconds
append_gn_df took 118.474504948 seconds
append_gn_df took 111.802592039 seconds
append_gn_df took 343.274796009 seconds
append_gn_df took 226.170608044 seconds
append_gn_df took 89.4556779861 seconds
append_gn_df took 117.234147072 seconds
chunk_gn_df took 1403.17566109 seconds


In [37]:
full_songs12999 = chunk_gn_df(songs, 12000, 13000)

append_gn_df took 133.29369092 seconds
append_gn_df took 151.5761621 seconds
append_gn_df took 92.5917768478 seconds
append_gn_df took 149.255569935 seconds
append_gn_df took 146.41320014 seconds
append_gn_df took 195.517437935 seconds
append_gn_df took 91.819890976 seconds
append_gn_df took 104.841881037 seconds
append_gn_df took 82.6004371643 seconds
append_gn_df took 85.0309300423 seconds
chunk_gn_df took 1234.32447505 seconds


In [39]:
full_songs13999 = chunk_gn_df(songs, 13000, 14000)

append_gn_df took 110.27683115 seconds
append_gn_df took 92.056000948 seconds
append_gn_df took 91.6379680634 seconds
append_gn_df took 92.28464818 seconds
append_gn_df took 109.537946939 seconds
append_gn_df took 99.7420189381 seconds
append_gn_df took 90.7187151909 seconds
append_gn_df took 99.281733036 seconds
append_gn_df took 104.241221905 seconds
Must query with at least one field (artist, album, track, toc)
append_gn_df took 125.22577405 seconds
chunk_gn_df took 1016.37598181 seconds


In [41]:
full_songs14999 = chunk_gn_df(songs, 14000, 15000)

append_gn_df took 102.279052973 seconds
append_gn_df took 149.364710808 seconds
append_gn_df took 98.630658865 seconds
append_gn_df took 93.2556109428 seconds
append_gn_df took 138.696675062 seconds
append_gn_df took 105.947779894 seconds
append_gn_df took 84.4272220135 seconds
append_gn_df took 91.7955269814 seconds
append_gn_df took 104.516112804 seconds
append_gn_df took 97.0090000629 seconds
chunk_gn_df took 1067.31919789 seconds


In [43]:
full_songs15999 = chunk_gn_df(songs, 15000, 16000)

append_gn_df took 88.1784000397 seconds
append_gn_df took 186.100250006 seconds
append_gn_df took 133.599490166 seconds
append_gn_df took 106.875953197 seconds
append_gn_df took 92.7605168819 seconds
append_gn_df took 117.334963083 seconds
append_gn_df took 92.7208259106 seconds
append_gn_df took 80.0384938717 seconds
append_gn_df took 82.366686821 seconds
append_gn_df took 66.072042942 seconds
chunk_gn_df took 1047.44817519 seconds


In [45]:
full_songs16999 = chunk_gn_df(songs, 16000, 17000)

append_gn_df took 68.4800598621 seconds
append_gn_df took 89.0547070503 seconds
append_gn_df took 76.2554938793 seconds
append_gn_df took 72.9623069763 seconds
append_gn_df took 83.4721598625 seconds
append_gn_df took 83.3267250061 seconds
append_gn_df took 68.4833159447 seconds
append_gn_df took 75.4022490978 seconds
append_gn_df took 69.1183638573 seconds
append_gn_df took 70.489814043 seconds
chunk_gn_df took 758.437319994 seconds


In [47]:
full_songs17999 = chunk_gn_df(songs, 17000, 18000)

append_gn_df took 65.9987528324 seconds
append_gn_df took 81.4038321972 seconds
append_gn_df took 76.6291849613 seconds
append_gn_df took 77.9058899879 seconds
append_gn_df took 75.1371262074 seconds
append_gn_df took 72.6422510147 seconds
append_gn_df took 78.9188940525 seconds
append_gn_df took 71.4597859383 seconds
append_gn_df took 68.2057061195 seconds
append_gn_df took 79.3313319683 seconds
chunk_gn_df took 749.036851168 seconds


In [49]:
full_songs18999 = chunk_gn_df(songs, 18000, 19000)

append_gn_df took 134.369413853 seconds
append_gn_df took 71.921598196 seconds
append_gn_df took 82.494289875 seconds
append_gn_df took 80.0985100269 seconds
append_gn_df took 85.3357758522 seconds
append_gn_df took 81.140155077 seconds
append_gn_df took 82.5929679871 seconds
append_gn_df took 71.8430140018 seconds
Must query with at least one field (artist, album, track, toc)
append_gn_df took 77.491494894 seconds
append_gn_df took 73.9180419445 seconds
chunk_gn_df took 842.593214035 seconds


In [51]:
full_songs19999 = chunk_gn_df(songs, 19000, 20000)

append_gn_df took 79.8324451447 seconds
append_gn_df took 74.6913099289 seconds
append_gn_df took 75.8829219341 seconds
append_gn_df took 74.6354169846 seconds
append_gn_df took 69.8141829967 seconds
append_gn_df took 72.1624469757 seconds
append_gn_df took 93.358782053 seconds
append_gn_df took 71.6024250984 seconds
append_gn_df took 69.3719408512 seconds
append_gn_df took 70.4650909901 seconds
chunk_gn_df took 753.183333158 seconds


In [53]:
full_songs20999 = chunk_gn_df(songs, 20000, 21000)

append_gn_df took 75.2547709942 seconds
append_gn_df took 94.6666181087 seconds
append_gn_df took 68.4411761761 seconds
append_gn_df took 79.5312569141 seconds
append_gn_df took 78.9820671082 seconds
append_gn_df took 77.2299990654 seconds
append_gn_df took 78.7224521637 seconds
append_gn_df took 73.3751819134 seconds
append_gn_df took 63.038287878 seconds
append_gn_df took 78.0099339485 seconds
chunk_gn_df took 768.634967089 seconds


In [55]:
full_songs21999 = chunk_gn_df(songs, 21000, 22000)

append_gn_df took 76.02511096 seconds
append_gn_df took 78.6484959126 seconds
append_gn_df took 78.8567829132 seconds
append_gn_df took 75.5170691013 seconds
append_gn_df took 74.2776551247 seconds
append_gn_df took 77.606345892 seconds
append_gn_df took 75.952696085 seconds
append_gn_df took 76.0307929516 seconds
append_gn_df took 93.7007489204 seconds
append_gn_df took 66.8212759495 seconds
chunk_gn_df took 774.803480864 seconds


In [57]:
full_songs22999 = chunk_gn_df(songs, 22000, 23000)

append_gn_df took 71.0597381592 seconds
append_gn_df took 71.5488610268 seconds
append_gn_df took 79.8974308968 seconds
append_gn_df took 77.8238909245 seconds
append_gn_df took 74.2301619053 seconds
append_gn_df took 79.7944989204 seconds
append_gn_df took 77.3801560402 seconds
append_gn_df took 74.3855559826 seconds
append_gn_df took 75.037225008 seconds
append_gn_df took 90.1520750523 seconds
chunk_gn_df took 772.704972029 seconds


In [59]:
full_songs23999 = chunk_gn_df(songs, 23000, 24000)

append_gn_df took 80.5861279964 seconds
append_gn_df took 76.774463892 seconds
append_gn_df took 67.6875209808 seconds
append_gn_df took 80.0155110359 seconds
append_gn_df took 76.2862389088 seconds
append_gn_df took 76.028192997 seconds
append_gn_df took 84.122674942 seconds
append_gn_df took 67.9391460419 seconds
append_gn_df took 73.0469720364 seconds
append_gn_df took 71.1367211342 seconds
chunk_gn_df took 755.017567873 seconds


In [61]:
full_songs24999 = chunk_gn_df(songs, 24000, 25000)

append_gn_df took 75.6429240704 seconds
append_gn_df took 89.7353980541 seconds
append_gn_df took 81.5058760643 seconds
append_gn_df took 83.1520318985 seconds
append_gn_df took 75.0766859055 seconds
append_gn_df took 78.1087810993 seconds
append_gn_df took 73.210572958 seconds
append_gn_df took 78.4695570469 seconds
append_gn_df took 71.3715658188 seconds
append_gn_df took 67.5516898632 seconds
chunk_gn_df took 775.238777161 seconds


In [ ]:
full_songs25999 = chunk_gn_df(songs, 25000)

In [88]:
def load_pickles(start, stop):
    df = pd.DataFrame([])
    for i in range(start, stop, 1000):
        df = df.append(pd.read_pickle('data/full_songs_df_'+str(i)+'.pkl'), ignore_index=True)
    return df

In [89]:
full_songs_most = load_pickles(999, 24999)

In [92]:
full_songs_all = full_songs_most.append(pd.read_pickle('data/full_songs_df_25899.pkl'), ignore_index=True)

### Load Pickle Point

In [94]:
full_songs_all.to_pickle('data/master_songs.pkl')

In [4]:
full_songs_all = pd.read_pickle('data/master_songs.pkl')

In [6]:
full_songs_all.head(10)

,artist,explicit,id,popularity,title,energy,liveness,tempo,speechiness,acousticness,...,artist_type_2,genre_1,genre_2,genre_3,gnid,mood_1,mood_2,tempo_1,tempo_2,tempo_3
0,Nirvana,False,3FUsMXBxA4V7eUwQ7B0HQO,48,Love Buzz,0.885,0.169,138.153,0.0354,0.000032,...,Male Group,Alternative & Punk,Alternative,Grunge,5007273-06983AE0CA94A42592A86F999A523B41,Brooding,Alienated / Brooding,Medium Tempo,Medium,60s
1,The Presidents Of The United States Of America,False,2JdzB67NvIa90K4gEZPLeX,58,Lump,0.873,0.165,142.726,0.0386,0.004680,...,Male Group,Alternative & Punk,Alternative,Alternative Pop,7176919-C893DAE82E18906A0C45F2D6651DF095,Aggressive,Aggressive Power,Fast Tempo,Fast,140s
2,Stanton Warriors,False,0zXLMZUCYZCx8Bq4jCr174,30,So Sweet,0.936,0.118,127.997,0.0425,0.000531,...,Male Duo,Electronica,Techno,Breakbeat,542968754-3A3DE9DCC8AACF9DE2F055187073DCA2,Energizing,Abstract Beat,Medium Tempo,Medium Fast,120s
3,Natalie Marchenko,False,14w7n7gi635VCb5f7t2OQ5,37,License To Chill - Pacha Sax Lounge RMX,0.511,0.24,171.963,0.0302,0.262000,...,NaN,Pop,Easy Listening,Other Easy Listening,212605678-5823A53026929EE7380477A9D3DCD697,NaN,NaN,NaN,NaN,NaN
4,Robyn,False,01NJd6s7Kyn6NSsPp503sh,28,Be Mine! - Live At The Cherrytree House,0.249,0.114,116.599,0.0451,0.955000,...,Female,Electronica,Electronica Mainstream,Pop Electronica,176433014-A2FF98AC415031B8A67464F3DED3BC22,Romantic,Heartfelt Passion,Medium Tempo,Medium,70s
5,Pearl Jam,False,6qgpO647NPBUvBkm56vV7d,5,Black,0.548,0.179,76.020,0.0287,0.294000,...,Male Group,Alternative & Punk,Alternative,Grunge,17527306-1F01011247D0A98978073E66BF4D5597,Brooding,Evocative / Intriguing,Medium Tempo,Medium,70s
6,Sam Hunt,False,3wx2kQWPn9p5UppQbNhPAk,67,Leave The Night On,0.953,0.349,171.971,0.0624,0.099600,...,Male,Traditional,Country,Contemporary Country,459278920-F9806BD4AFF4B5DC038D6D65E70C0B6C,Yearning,Sensitive / Exploring,Medium Tempo,Medium,80s
7,George Carlin,False,5DuxYjcuebFPontojL1NEE,0,War Pictures,0.546,0.147,72.940,0.961,0.921000,...,Male,Other,Comedy,Comedy,4965187-7F868B4D3F7D16781A583E8E222EF876,Other,NaN,Slow Tempo,Static,0
8,Kaya May,False,4eg3wsiTrPa7KpwVai2VY7,56,"Hands to Myself, Love Yourself, Perfect (Acous...",0.445,0.113,110.056,0.05,0.057500,...,NaN,Pop,Western Pop,Pop Vocal,604743990-4B9A5237E117F5CE9E8506362D53C1D9,Brooding,Dreamy Brooding,Medium Tempo,Medium Slow,50s
9,Notaker,False,341ShRaeysVdxLetlI391K,56,Infinite,0.669,0.108,99.987,0.0483,0.093700,...,NaN,Electronica,Dance & Club,Club Dance,625141650-9A245193045FB288D1785D87C9D94308,Excited,Euphoric Energy,Medium Tempo,Medium Fast,100s


In [105]:
full_songs_all.count(axis=0)

artist              24567
explicit            24567
id                  24567
popularity          24567
title               24567
energy              24567
liveness            24561
tempo               24567
speechiness         24531
acousticness        24567
instrumentalness    24566
time_signature      24531
danceability        24531
key                 24567
duration_ms         24567
loudness            24567
valence             24529
mode                24567
artist_era_1        20782
artist_era_2         8151
artist_origin_1     20791
artist_origin_2     20769
artist_origin_3     15260
artist_origin_4     10986
artist_type_1       20785
artist_type_2       20744
genre_1             24354
genre_2             24354
genre_3             24354
gnid                24567
mood_1              21500
mood_2              21059
tempo_1             21499
tempo_2             21499
tempo_3             21499
dtype: int64